In [1]:
import pandas as pd

from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario

ModuleNotFoundError: No module named 'covid_xprize'

# Scenario generator

## Latest data

In [ ]:
DATA_FILE = "tests/fixtures/OxCGRT_latest.csv"
latest_df = get_raw_data(DATA_FILE, latest=True)

# Scenario: historical IP until 2020-09-30
Latest historical data, truncated to the specified end date

In [ ]:
start_date_str = None
end_date_str = "2020-09-30"
countries = None
output_file = "data/2020-09-30_historical_ip.csv"

In [ ]:
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries, scenario="Historical")

In [ ]:
scenario_df[scenario_df.CountryName == "France"].Date.max()

In [ ]:
truncation_date = pd.to_datetime(end_date_str, format='%Y-%m-%d')
scenario_df = scenario_df[scenario_df.Date <= truncation_date]

In [ ]:
scenario_df.tail()

In [ ]:
scenario_df.to_csv(output_file, index=False)

# Scenario: frozen NPIs
Latest historical data + frozen NPIS between last known date and end of Januaray 2021 for India and Mexico

In [ ]:
start_date_str = "2021-01-01"
end_date_str = "2021-01-31"
countries = ["India", "Mexico"]

In [ ]:
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries, scenario="Freeze")

In [ ]:
len(scenario_df)

In [ ]:
scenario_df.CountryName.unique()

In [ ]:
scenario_df.tail()

## Save

In [ ]:
hist_file_name = "data/future_ip.csv"
hist_file_name

In [ ]:
scenario_df.to_csv(hist_file_name, index=False)